### Analisi Dati Facebook

1. pip install facebook-sdk: https://github.com/mobolic/facebook-sdk, https://facebook-sdk.readthedocs.io/en/latest/
    
2. Tool di Esplorazioni API Graph: https://developers.facebook.com/tools/explorer/145634995501895/#_=_
    
3. ID Pagina Facebook: http://findmyfbid.com/

In [4]:
import facebook
import os
import requests

# Leggo Configuration File
config = {}
config_path = os.path.join(os.environ['HOME'], 'opendata')
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
token=config['TOKEN_FB']

# Utilizzo token generoato manualmente con la mia utenza sul Tool di Esplorazioni API Graph
graph = facebook.GraphAPI(access_token=token, version='2.7')

In [5]:
# TEST

# Esempio di chiamata su Tool Esplorazione: me?fields=id,name -> Result è un Dict (casomai Dataframe Pandas)
info = graph.get_object(id='me')
info

{'id': '10153605453803372', 'name': 'Andrea Cesarini'}

In [6]:
# PAGINA FACEBOOK

# Recupero ID della Pagina di Salvini (https://www.facebook.com/salviniofficial/?fref=ts) da findmyfbid 
# Esempio di chiamata: 252306033154?fields=id,name,birthday
info_s = graph.get_object(id='252306033154', fields='name,fan_count,posts')

In [7]:
# NUMERO FAN

# Recupero il numero di Fan (sono quelli che mettono mi piace, non è detto che siano quelli che lo seguono in
# quanto uno mette mi piace anche solo per seguirlo)
num_fan = info_s['fan_count']
num_fan

1694775

In [29]:
# ULTIMO POST PUBBLICATO

# Recupero id dell'ultimo post pubblicato (intreccio di Dict e List) -> Per ora analizzo il penultimo post
id_post = info_s['posts']['data'][1]['id']
id_post

'252306033154_10154348485508155'

In [30]:
# DETTAGLI POST
post_mess = graph.get_object(id=id_post, fields='message')
post_mess

{'id': '252306033154_10154348485508155',
 'message': 'Immigrati oggi in piazza: "Vogliamo soldi e non vogliamo confini".\n#aveterottolepalle'}

In [31]:
# CHI METTE MI PIACE AI POST DI SALVINI

likes = graph.get_connections(id=id_post, connection_name='likes')

# Creo lista con gli ID di tutti quelli che hanno messo mi piace al post
all_likes = []
while(True):
    try:
        for like in likes['data']:
            #Recupero chi mette like
            all_likes.append(like['id'])
        likes=requests.get(likes['paging']['next']).json()
    except KeyError:
        break
len(all_likes)

2444

### Con immensa tristezza, non si possono analizzare i dati pubblici delle persone che hanno messo like ai post (es. sesso, distr geografica, educazione). Si può leggere solo il Totale dei likes

# 1. Distribuzione likes ai posts in base alla data dei post (aggregati per anno / mese)
## Crescita social delle pagine politiche -> In base alla data, legarle ad avvenimenti

In [43]:
info_s

{'fan_count': 1694775,
 'id': '252306033154',
 'name': 'Matteo Salvini',
 'posts': {'data': [{'created_time': '2016-12-18T19:46:07+0000',
    'id': '252306033154_10154348697478155',
    'message': 'Mia intervista di stamane al "Giornale di Sicilia". Sovranità monetaria, difesa dei confini, stop agli euroidioti che distruggono pesca, agricoltura e lavoro, e la Sicilia tornerà ad essere una capitale mondiale del turismo, dell’arte, della cultura, del made in Italy, non l\'isola dei clandestini!'},
   {'created_time': '2016-12-18T18:05:00+0000',
    'id': '252306033154_10154348485508155',
    'message': 'Immigrati oggi in piazza: "Vogliamo soldi e non vogliamo confini".\n#aveterottolepalle'},
   {'created_time': '2016-12-18T16:34:54+0000',
    'id': '252306033154_10154348370848155',
    'message': 'La Boldrini: "Senza migranti Italia piena di anziani senza pensione".\nMa ancora parla? #boldriniclandestina'},
   {'created_time': '2016-12-18T12:37:09+0000',
    'id': '252306033154_101543478

# 2. Sentiment Analysis sui Commenti dei Post - Pros vs Cons (DOPO..)
con occhio all'ironia

In [41]:
# COMMENTI POST
commenti = graph.get_connections(id=id_post, connection_name='comments')

In [42]:
# TUTTI GLI ID DELLE PERSONE CHE HANNO COMMENTATO
allcommenti = []
while(True):
    try:
        for commento in commenti['data']:
            #Recupero chi commenta (posso recuperare anche il commento)
            #Sentiment analysis chi commenta e come commenta distributo geograficamente tra positivi e negativi
            allcommenti.append(commento['from'])
        commenti=requests.get(commenti['paging']['next']).json()
    except KeyError:
        break
len(allcommenti)

957